In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer



Vamos a analizar los textos de las críticas dejadas por usuarios en el sitio metacritic.com para el juego "The Legend of Zelda: Breath of the Wild ", y apuntar a discernir entre aquellas donde la nota final se encuentra por debajo o por encima de un cierto umbral. Si el modelo es exitoso, dado un texto, podría predecir correctamente si el mismo corresponde a una nota mayor o menor al umbral. 


Primero, leemos los datos de un archivo externo y los "limpiamos". 

In [3]:
reviewsDf = pd.read_csv('zeldareviews.csv', index_col = 0).drop(['name'], axis = 1)
reviewsDf.head()

,score,text
0,10,\nSimply Amazing. Nuff said. This is the best ...
1,10,"I love how this game is so good, that triggerd..."
2,10,\nSimply breathtaking and stunning. Nintendo h...
3,10,\nEste juego es la perfeccion en cuanto a jueg...
4,10,"\nStunning in every way, While everyone though..."


In [4]:
reviewTexts = reviewsDf['text']
reviewTexts = reviewTexts.apply(lambda x: x.replace('\r', '').replace('\n', ''))

Dividimos en un conjunto de entrenamiento y uno de testeo (con una proporción de 75:25), y definimos los labels: van a ser positivo si el puntaje es mayor a 8 (es decir, un 9 o un 10), y negativo en caso contrario 

In [8]:
reviewsList = [text for text in reviewTexts]
text_train = reviewsList[:int(len(reviewsList)*0.75)]
y_train = [score > 8 for score in reviewsDf['score']][:int(len(reviewsList)*0.75)]
text_test = reviewsList[int(len(reviewsList)*0.75)+1:]
y_test = [score > 8 for score in reviewsDf['score']][int(len(reviewsList)*0.75)+1:]
scores = [score for score in reviewsDf['score']][int(len(reviewsList)*0.75)+1:]

Comenzamos el proceso llevando el conjunto de palabras a un formato numérico para poder trabajar.

In [9]:
vect = CountVectorizer(min_df = 1) #CountVectorizer lleva el texto a nros
vect.fit(text_train)
print("Vocabulary size: {}".format(len(vect.vocabulary_)))
X_train = vect.transform(text_train)
print("bag_of_words: {}".format(repr(X_train)))

Vocabulary size: 19406
bag_of_words: <3495x19406 sparse matrix of type '<class 'numpy.int64'>'
	with 258867 stored elements in Compressed Sparse Row format>


Ahora buscamos, para un modelo de regresión logística, el valor de C que lleva a un menor error, usando gridsearchCV de scikit. 

In [10]:
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
grid = GridSearchCV(LogisticRegression(max_iter = 1000), param_grid, cv=5)
grid.fit(X_train, y_train)
print("Best cross-validation score: {:.2f}".format(grid.best_score_))
print("Best parameters: ", grid.best_params_)

Best cross-validation score: 0.89
Best parameters:  {'C': 1}


Para ver cuántos errores y de qué tipo presenta el modelo, estudiamos la matriz de confusión. 

In [11]:

predicted_labels = cross_val_predict(LogisticRegression(max_iter = 1000), X_train, y_train, cv=5)
confusion = confusion_matrix(y_train, predicted_labels)

print("Confusion Matrix:")
print(confusion)

Confusion Matrix:
[[ 385  275]
 [ 119 2716]]


Preparamos el modelo que se eligió como mejor, y lo entrentamos con la data de entrenamiento.

Luego, generamos las predicciones del conjunto de testeo y tomamos las métricas de interés.

In [12]:
logreg = LogisticRegression(max_iter = 1000, C = 1) 
logreg.fit(X_train, y_train)
X_test = vect.transform(text_test)
y_pred = logreg.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}\n")
print("Classification Report:\n", report)
print("\n Confusion Matrix: ")
print(confusion)

Accuracy: 0.89

Classification Report:
               precision    recall  f1-score   support

       False       0.49      0.34      0.40       125
        True       0.92      0.96      0.94      1039

    accuracy                           0.89      1164
   macro avg       0.71      0.65      0.67      1164
weighted avg       0.88      0.89      0.88      1164


 Confusion Matrix: 
[[ 42  83]
 [ 43 996]]


--------

El rendimiento del modelo es particularmente malo a la hora de clasificar falsos, es decir, reviews "negativas". Esto se da porque subimos el umbral para los casos "negativos" a un valor de 8, lo cual no llega a ser suficientemente distinguible de los valores "positivos" (puntaje de 9 o 10) como para que el modelo lo detecte (hasta para un humano resulta difícil diferenciar un 8 de un 9). El problema es que bajar el límite nos dejaría menos data para trabajar, ya que el número de casos negativos sería muy bajo.

Si bien no es un buen modelo para diferencias tan sutiles, si lo es para detectar reviews muy positivas (de 9 o 10), tienendo un 92% de precisión y un 96% de recall, es decir, estamos detectando el 96% de los casos deseados. 

Este modelo sería entonces de utilidad en caso de que se quiera identificar principalmente a las reviews muy positivas, por ejemplo, para medir el interés en contenido adicional relacionado al juego.


Otra alternativa es subir el umbral, y diferenciar los 10 de el resto. Esto aumentaría la precisión de detección de los falsos, a costa de bajar la precisión de los verdaderos. 

Sería interesante probar el modelo en reviews de otros juegos. Lo más seguro es que la performance sea bastante pobre, principalmente porque los vocabularios utilizados difieren, y lo que es positivo para un juego es negativo para otro.

Si se quisiera generalizar, se debería utilizar data de entrenamiento perteneciente a un conjunto lo más diverso posible de juegos. 

-----------